# Lesson 3: Neural Networks and Loss Functions



nn.Module, specifying a loss function, MNIST classification.

Intro to CNNs

Inspecting a modern network

Bonus: Working with pretrained models

Bonus: Intro to Dataloaders

### Exercise: Fine-tune a pretrained model on a new classification task